## Stochastic Thinking and Random Walks
随机思维

The world may or may not be inherently unpredicatable  
But our lack of knowledge does not allow us to make accurate predictions  
Therefore we might as well threat the world as inherently unpredictable  
**Predictive nondeterminism**

### Implementing a Randomo Process

In [1]:
import random 

def rollDie():
    '''returns a random int between 1 and 6'''
    return random.choice([1,2,3,4,5,6])

def testRoll(n=10):
    result = ''
    for i in range(n):
        result = result + str(rollDie())
    print(result)

testRoll()

4551526651


### A Simulation of Die Rolling

In [5]:
def runSim(goal, numTrials, txt):
    total = 0
    for i in range(numTrials):
        result = ''
        for j in range(len(goal)):
            result += str(rollDie())
        if result == goal:
            total += 1
    print('Actual probability of',txt,'=',round(1/(6**len(goal)),8))
    estProbability = round(total/numTrials,8)
    print('Estimated Probability of',txt,'=',round(estProbability,8))
    
runSim('11111',1000000,'11111')

Actual probability of 11111 = 0.0001286
Estimated Probability of 11111 = 0.000112


### Approximating Using a Simulation

In [9]:
def sameDate(numPeople, numSame):
    possibleDates = range(366)
    birthdays = [0]*366
    for p in range(numPeople):
        birthDate = random.choice(possibleDates)
        birthdays[birthDate] += 1
    return max(birthdays) >= numSame

In [10]:
def birthdayProb(numPeople, numSame, numTrials):
    numHits = 0
    for t in range(numTrials):
        if sameDate(numPeople, numSame):
            numHits += 1
    return numHits/numTrials

import math

for numPeople in [10, 20, 40, 100]:
    print('For', numPeople,
          'est. prob. of a shared birthday is',
          birthdayProb(numPeople, 2, 10000))
    numerator = math.factorial(366)
    denom = (366**numPeople)*math.factorial(366-numPeople)
    print('Actual prob. for N = 100 =',
          1 - numerator/denom)

For 10 est. prob. of a shared birthday is 0.116
Actual prob. for N = 100 = 0.1166454118039999
For 20 est. prob. of a shared birthday is 0.41
Actual prob. for N = 100 = 0.4105696370550831
For 40 est. prob. of a shared birthday is 0.895
Actual prob. for N = 100 = 0.89054476188945
For 100 est. prob. of a shared birthday is 1.0
Actual prob. for N = 100 = 0.9999996784357714


### But All Dates Are Not Equally Likely

![birthdaydate](./img/BirthdayDate.png)

## Random Walks

#### Random walks are important in many domains
Understanding the stock market(maybe)  
Modeling diffusion processes  
Etc.
#### Good illustration of how to use simulation to understand things